In [224]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [225]:
from enc_dec import *

In [226]:
gpuid = -1

In [227]:
xp = cuda.cupy if gpuid >= 0 else np

In [228]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [229]:
text_data.keys()

dict_keys(['train', 'dev', 'test'])

In [237]:
model = SpeechEncoderDecoder(120, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)

In [238]:
sp_fil = np.load(os.path.join(speech_dir, "041.001"+speech_extn))

In [239]:
t_sp_fil = xp.expand_dims(sp_fil, 1)

In [240]:
t_sp_fil.shape, t_sp_fil[0].shape, sp_fil[0].shape

((1168, 1, 120), (1, 120), (120,))

In [194]:
np.expand_dims(sp_fil[:5],0).shape

(1, 5, 120)

In [195]:
sp_fil.shape

(1168, 120)

In [196]:
t_lstm = L.LSTM(sp_fil.shape[1], 200)

In [149]:
A = t_lstm(sp_fil)

In [150]:
(A.data==t_lstm.h.data).all()

True

In [151]:
t_lstm(sp_fil[:5]).shape

(5, 200)

In [206]:
len(sp_fil), t_lstm.h.shape

(1168, (1, 200))

In [245]:
out_states = model.feed_pyramidal_lstm(t_sp_fil[:10], lstm_layer=model.lstm_enc[0], scale=2, train=False)

In [246]:
out_states.shape

(5, 1024)

In [242]:
out_states = feed_pyramidal_lstm(sp_fil[:10], scale=2, train=False)

(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)


In [205]:
out_states.shape

(5, 400)

In [200]:
def feed_pyramidal_lstm(feat_in, scale, train):
    xp = cuda.cupy if gpuid >= 0 else np

    # create empty array to store the output
    # the output is scaled by the scale factor
    n_out_states = len(feat_in) // scale
    out_dim = t_lstm.state_size
    out_states = xp.empty((0, out_dim*scale), dtype=xp.float32)

    # feed each input from the sequence
    for i in range(0, n_out_states):
        # lateral_states = xp.empty((0,out_dim), dtype=xp.float32)
        lateral_states = t_lstm(xp.expand_dims(feat_in[(i*scale)],0))
        for j in range(1, scale):
            out = t_lstm(xp.expand_dims(feat_in[(i*scale)+j],0))
            print(out.shape)
            print(lateral_states.shape)
            lateral_states = F.concat((lateral_states, out), axis=1)
        # concatenate and append lateral states into out states
        out_states = F.concat((out_states, lateral_states), axis=0)

    return out_states

In [91]:
feat_in = np.random.rand(10, 40).astype(np.float32)
scale = 2
n_out_states = len(feat_in) // scale
print(n_out_states)

5


In [93]:
haha=np.empty(30, dtype=np.float32)

In [94]:
haha.shape

(30,)

In [92]:
for i in range(0, n_out_states):
    for j in range(0, scale):
        print((i*scale)+j)

0
1
2
3
4
5
6
7
8
9


In [ ]:
for i in range(0, len(feat_in), scale):
    lateral_states = xp.empty((scale, out_dim), dtype=xp.float32)
    for j in range(i, i+scale):
        lateral_states[j-i] = self[lstm_layer](feat_in[j])
    # concatenate and append lateral states into out states
    n_out_states[]

In [53]:
for i in range(0,10,2):
    for j in range(i, i+2):
        print(j, end=",")
    print("")

0,1,
2,3,
4,5,
6,7,
8,9,


In [ ]:
for r in range(len(sp_fil)):
    